In [1]:
from __future__ import print_function
import tensorflow as tf
import keras.models 
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Activation, Conv2D, GlobalAveragePooling2D, BatchNormalization
from keras.optimizers import SGD
from keras import backend as K
from keras.models import Model
from keras.layers.core import Lambda
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
import numpy as np

Using TensorFlow backend.


In [2]:
# Prepare data
split_rate = 0.1
def split_validation(train, split_rate):
    validation_index = int(len(train) * split_rate);
    validation = train[-validation_index:];
    train = train[:-validation_index];
    return train, validation
batch_size = 32 
num_classes = 10
epochs = 500
data_augmentation = True
# Load cifar10 data and split train data to get validation data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_validation = split_validation(x_train, split_rate)
y_train, y_validation = split_validation(y_train, split_rate)
x_train = x_train.astype('float32')
x_validation = x_validation.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_validation /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_validation = keras.utils.to_categorical(y_validation, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (45000, 32, 32, 3)
45000 train samples
10000 test samples


In [35]:
model = Sequential()

model.add(Conv2D(96, (3, 3), padding = 'same', input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(Conv2D(96, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(96, (3, 3), padding='same', strides = (2,2)))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(Conv2D(192, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(Conv2D(192, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(192, (3, 3), padding='same', strides = (2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Conv2D(192, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(Conv2D(192, (1, 1), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(10, (1, 1), padding='same'))


model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))

print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_49 (Conv2D)           (None, 32, 32, 96)        2688      
_________________________________________________________________
activation_38 (Activation)   (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
activation_39 (Activation)   (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 16, 16, 96)        83040     
_________________________________________________________________
dropout_12 (Dropout)         (None, 16, 16, 96)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 96)        384       
__________

In [36]:
def lr_schedule(epoch):
    # Learning Rate Schedule
    lr = 0.05
    if epoch > 200:
        lr *= 0.1
    if epoch > 250:
        lr *= 0.1
    if epoch > 300:
        lr *= 0.1
    
    print('Learning rate: ', lr)
    return lr

In [37]:
filepath="AllConv_model.h5"
lr_scheduler = LearningRateScheduler(lr_schedule)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', 
                                         histogram_freq=0, 
                                         write_graph=True, 
                                         write_images=True)

callbacks_list = [checkpoint, tbCallBack, lr_scheduler]

In [ ]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) 

datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs, 
                    validation_data=(x_validation, y_validation), 
                    callbacks=callbacks_list, verbose=1)

In [6]:
model = keras.models.load_model('model/AllConv_model.h5');
model.evaluate(x=x_test, y=y_test, verbose=1)

10000/10000 [==============================] - 2s 152us/step


[0.4767506147934124, 0.912]